# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 


**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [ ]:
import numpy as np
import pandas as pd
import re
import nltk

from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords as nltk_stopwords

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
from sklearn.pipeline import Pipeline

import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [ ]:
data = pd.read_csv('/datasets/toxic_comments.csv')
data.info()

In [ ]:
data.head()

проверим на дисбаланс классов целевой признак

In [ ]:
data['toxic'].value_counts()

посчитаем отношение 

In [ ]:
class_ratio = data['toxic'].value_counts()[0] / data['toxic'].value_counts()[1]
class_ratio

выполним лемматизацию

In [ ]:
# pandarallel.initialize(progress_bar=True)
# lemmatizer = WordNetLemmatizer()

# def get_wordnet_pos(word):
#     """Map POS tag to first character lemmatize() accepts"""
#     tag = nltk.pos_tag([word])[0][1][0].upper()
#     tag_dict = {"J": wordnet.ADJ,
#                 "N": wordnet.NOUN,
#                 "V": wordnet.VERB,
#                 "R": wordnet.ADV}
#     return tag_dict.get(tag, wordnet.NOUN)

# def lemmatize(text):
#     text = text.lower()  
#     words = word_tokenize(text)  
#     tagged_words = pos_tag(words) 
#     lemmatized_words = [lemmatizer.lemmatize(word, get_wordnet_pos(tag)) for word, tag in tagged_words]  
#     cleared_text = re.sub(r'[^a-zA-Z]', ' ', " ".join(lemmatized_words)).strip() 
#     return " ".join(cleared_text.split())

# data['lemm_text'] = data['text'].parallel_apply(lemmatize)
# data = data.drop(['text'], axis=1)

In [ ]:
lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    text = text.lower()
    lemm_text = "".join(lemmatizer.lemmatize(text))
    cleared_text = re.sub(r'[^a-zA-Z]', ' ', lemm_text) 
    return " ".join(cleared_text.split())

data['lemm_text'] = data['text'].apply(lemmatize_text)

data = data.drop(['text'], axis=1)

## Обучение

Разделим данные на две выборки, с размером тестовой в 0.25

In [ ]:
data = data.drop('Unnamed: 0', axis=1)

In [ ]:
X = data.drop(['toxic'], axis=1)
y = data['toxic']

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size = 0.25,
                                                    random_state = 42,
                                                    stratify=y)

In [ ]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

count_tf_idf = TfidfVectorizer(stop_words=stopwords)

#X_train = count_tf_idf.fit_transform(X_train['lemm_text'].values.astype('U'))
#X_test = count_tf_idf.transform(X_test['lemm_text'].values.astype('U'))

#print(X_train.shape)
#print(X_test.shape)

Теперь обучим модель логистической регрессии

In [ ]:
%%time
model_lr = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words=stopwords)),
    ('clf', LogisticRegression())
])
сv_f1_lr = (cross_val_score(model_lr, 
                              X_train['lemm_text'], 
                              y_train, 
                              cv=2, 
                              scoring='f1').mean())
print('Качество метрики f1 линейной регрессии при кросс валидации', сv_f1_lr)

In [ ]:
# %%time
# classificator = CatBoostClassifier(verbose=False, iterations=250)
# cv_f1_CBC = cross_val_score(classificator,
#                             X_train, 
#                             y_train, 
#                             cv=2, 
#                             scoring='f1').mean()
# print('F1 на cv', cv_f1_CBC)

выполним уменьшение размера весов класса и обучим модель линейной регрессии

In [ ]:
%%time
dict_classes={0:1, 1:class_ratio}
model2 = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words=stopwords)),
    ('clf', LogisticRegression(class_weight=dict_classes))
])
f1_ballanced = cross_val_score(model2, 
                                X_train['lemm_text'], 
                                y_train, 
                                cv=2, 
                                scoring='f1').mean()
print('Качество метрики f1 линейной регрессии при кросс валидации с уменьшением классов', f1_ballanced)

значение метрики  f1 > 0.75, следовательно будем использовать эту модель

In [ ]:
hyperparams = [{'clf__C':[0.1, 1, 10],
                'clf__class_weight':[dict_classes]}]
clf = GridSearchCV(model2, hyperparams, scoring='f1',cv=2)
clf.fit(X_train['lemm_text'], y_train)
best_params = clf.best_params_
print(f'Лучшие параметры для модели: {best_params}')
print()
print(f'Лучшая оценка f1 для модели: {clf.best_score_}')

оценим качество нашей модели на тестовых данных

In [ ]:
best_model = clf.best_estimator_
y_pred = best_model.predict(X_test['lemm_text'])

f1 = f1_score(y_test, y_pred)
print(f'F1-score на тестовых данных: {f1}')

## Выводы

подготовили данные для обучения, устранили дисбаланс классов, сформировали обучающую и тестовую выборку, обучили две модели - 
CatBoost и LogisticRegression, по результатам кросс валидации лучшей оказалась модель логистической регрессии с качеством метрики f1 равной 0.76 на трейне и 0.77 на тесте